In [0]:
!pip install simpletransformers


In [2]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [0]:
!sh setup.sh


In [0]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import accuracy_score

In [0]:
def int_enc(values):
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    return(integer_encoded)

In [6]:
df = pd.read_csv('/content/drive/My Drive/CIPS LAB/sa-emotions/train_data.csv', delimiter=',')
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [0]:
df_bin = df.loc[(df['sentiment'] == 'sadness') | (df['sentiment'] == 'happiness')]

In [0]:
def cleaner(tes):
    REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
    REPLACE_U_NAME = re.compile("@[\S]+")
    REPLACE_DIGITS = re.compile("\d")
    REPLACE_W_SPACE = re.compile("_")
    tes["content"] = tes["content"].str.replace(REPLACE_NO_SPACE, '')
    tes["content"] = tes["content"].str.replace(REPLACE_U_NAME,'')
    tes["content"] = tes["content"].str.replace(REPLACE_DIGITS,'')
    tes["content"] = tes["content"].str.replace(REPLACE_W_SPACE,'')
    tes["content"] = tes["content"].str.lower()
    return tes

In [0]:
df_bin = cleaner(df_bin)
df_bin.head()

In [0]:
col_list = list(df_bin)
col_list[0], col_list[1] = col_list[1], col_list[0]
df_bin = df_bin.ix[:,col_list]

In [0]:
df_bin['sentiment'] = int_enc(np.array(df_bin['sentiment']))

In [12]:
df_bin.head()

,content,sentiment
1,layin n bed with a headache ughhhhwaitin on y...,1
2,funeral ceremonygloomy friday,1
6,i should be sleep but im not thinking about an...,1
8,charlene my love i miss you,1
9,im sorry at least its friday,1


In [0]:
X_train, X_test, y_train, y_test = train_test_split(df_bin['content'], df_bin['sentiment'],
                                                    stratify=df_bin['sentiment'], 
                                                    test_size=0.25)

In [16]:
print(len(X_train),len(X_test),len(X_train),len(y_test))

5860 1954 5860 1954


In [0]:
train_data_df = pd.concat([X_train, y_train], axis=1)
test_data_df = pd.concat([X_test, y_test], axis=1)

In [18]:
test_data_df.head()

,content,sentiment
8997,is sad because his tea and oatmeal are cold,1
1190,says im still dreaming about that nikon derrr ...,1
2617,wow what a sunny day just recovering some era...,0
23842,happy star wars day may the th be with you,0
12251,off to take a math quizon a fridaybooo,1


In [21]:
import pandas as pd

# Create a ClassificationModel
model = ClassificationModel('bert', 'bert-base-cased', num_labels=2, args={'reprocess_input_data': True, 'overwrite_output_dir': True,'learning_rate':1e-5, 'num_train_epochs': 5}) 

# Train the model
model.train_model(train_data_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_data_df,acc=accuracy_score)



Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:256: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.044088

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:113: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


...

Running loss: 0.089561Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 0.017853

Running loss: 1.445241

Running loss: 0.002437
Training of bert model complete. Saved to outputs/.
Features loaded from cache at cache_dir/cached_dev_bert_128_2_1954


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



{'mcc': 0.8765353030375278, 'tp': 1149, 'tn': 691, 'fp': 56, 'fn': 58, 'acc': 0.9416581371545547, 'eval_loss': 0.2743764496883567}
